In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
df_anon=pd.read_excel("Actividades de Seguimiento/Ati_anonimo.xlsx")
print(len(df_anon))
df_anon_unicos=df_anon.drop_duplicates(subset=['DNI','COMISION'],keep=False)
print(len(df_anon_unicos))


4326
3981


In [49]:
#ANALISIS DE LA ACTIVIDAD DE SEGUIMIENTO 1 AMBAS COMISIONES----------------------------------------------------------


df=pd.DataFrame(columns=['DNI','COMISION','CARRERA','PREGUNTA 1','PREGUNTA 2','PREGUNTA 3','PREGUNTA 4','PREGUNTA 5','PREGUNTA 6','PREGUNTA 7','PREGUNTA 8','PREGUNTA 9'])

df['DNI']=df_anon_unicos['DNI'].astype(int)
df['COMISION']=df_anon_unicos['COMISION'].astype(str)
df['CARRERA']=df_anon_unicos['Carrera-Nom'].astype(str)


archivos_en_carpeta=os.listdir(f"Actividades de Seguimiento/Actividad de Seguimiento 1")
for dni in df.DNI:
#for dni in [41449081]:
    com=df['COMISION'][df.DNI==dni].values[0]
    for archivo in archivos_en_carpeta:
        if str(com) in archivo:
            df_a1=pd.read_excel(f"Actividades de Seguimiento/Actividad de Seguimiento 1/{archivo}")
            mask=df_a1['Número de ID']==dni
            
            result = df_a1[mask].iloc[:,11:].values
            
            if result.shape[0] > 0:
                fila=result[0]
                fila_numeros=np.array([float(valor.replace(',', '.'))if valor.replace(',','.').replace('.','',1).isdigit() else np.nan for valor in result[0]])
                df.iloc[df.DNI==dni,3:] = fila_numeros
            # else: no data for this dni in this file, skip assignment

           
            








In [ ]:
#VISUALIZACION DE DATOS DE LA ACTIVIDAD DE SEGUIMIENTO 1

print("Datos de la Actividad de Seguimiento 1:")
print('')

print("Puntuacion promedio de cada pregunta:")
promedios=[]
topes=[]
for num in range(1,10 ):
    promedio = float(df[f'PREGUNTA {num}'].mean())
    promedios.append(round(promedio, 2))
    valor = num + 10
    # Extraer valor_max del nombre de la columna por ejemplo: 'P. 1 /0,75'
    nombre_columna = df_a1.columns[valor]
    if '/' in nombre_columna:
        valor_max = float(nombre_columna.split('/')[1].replace(',', '.'))
        topes.append(valor_max)
    
    print(f"PREGUNTA {num}  :  {promedio:.2f} de {valor_max:.2f}")

#Histograma de las puntuaciones de cada pregunta en relacion al valor maximo de la pregunta
razon=[]
preguntas=[]
print('')
print(f"Puntuacion promedio de cada pregunta en relacion a su valor maximo:")
for indice in range(1, 10):
     razon.append(promedios[indice - 1] / topes[indice - 1])
     lugar=indice+2
     preguntas.append(f'Pregunta {indice} ')
     
     print(f"PREGUNTA {indice} : {razon[indice - 1]:.2f} ")

print('')
plt.bar(preguntas, razon, color='red', edgecolor='red',alpha=0.7)
plt.suptitle('Puntuación promedio de cada pregunta en relacion a su valor máximo (Actividad de Seguimiento 1)')
plt.title('Comisiones M y B')

plt.ylabel('Puntuación promedio / Valor máximo')
plt.xticks(rotation=45)
plt.grid(True, axis='y', alpha=0.3)
plt.show()

In [ ]:
#ACTIVIDAD SE SEGUIMIENTO 1 POR CARRERA----------------------------------------------------------------
print('ACTIVIDAD DE SEGUIMIENTO 1 POR CARRERA (COMISIONES M Y B)------------------------------------------------------------------------------------------------')
print('')
for carrera in df['CARRERA'].unique():
    
    mask=df['CARRERA']==carrera
    filas_true=df[mask]

    print(f'CARRERA: {carrera}')


    if len(filas_true) <= 50:
        print(f"No se analiza esta carrera porque tiene {len(filas_true)} alumnos (menos de 51).")
        continue


    result = filas_true.iloc[:, 3:].values #con esto obtengo todas las filas que cumplen la condicion de la carrera
   
    
    print(f'Puntuación promedio de cada pregunta:')
    
        # Verificar que result no esté vacío
    if result.size > 0:
            result = result.astype(float)
            promedios = np.nanmean(result, axis=0)
            
            # Imprimir los promedios
          
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"PREGUNTA {i}: No hay datos válidos (todos son nan)")
                else:
                    print(f"PREGUNTA {i}: {promedio:.2f}")
            
            print('')
            print('Puntuación promedio de cada pregunta en relación a su valor máximo:')
            razones=[]
            preguntas=[]
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"Razón PREGUNTA {i}: No se puede calcular (sin datos válidos)")
                else:
                    razon = promedio / topes[i-1]
                    print(f"Razón PREGUNTA {i}: {razon:.2f}")
                    razones.append(razon)
                    preguntas.append(f"PREGUNTA {i}")
        
        # Generar gráfico de barras para las razones
            if razones:  # Solo generar el gráfico si hay razones válidas
                plt.figure(figsize=(10, 6))
                plt.bar(preguntas, razones, color='green',edgecolor='green',alpha=0.7)
                
                plt.ylabel('Razón (Promedio / Valor Máximo)')
                plt.title(f'Razones por Pregunta para la Carrera {carrera}')
                plt.suptitle('Actividad de Seguimiento 1 ')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            else:
                print("No hay razones válidas para generar el gráfico.")
            

    else:
        print("No hay filas para esta carrera.")

    print('')


            

            



In [16]:
#ANALISIS DE LA ACTIVIDAD DE SEGUIMIENTO 2 COMISIONES M Y B SEPARADAS---------------------------------------------------------


df2M=pd.DataFrame(columns=['DNI','COMISION','CARRERA','PREGUNTA 1','PREGUNTA 2','PREGUNTA 3','PREGUNTA 4','PREGUNTA 5','PREGUNTA 6','PREGUNTA 7','PREGUNTA 8'])
df2B=pd.DataFrame(columns=['DNI','COMISION','CARRERA','PREGUNTA 1','PREGUNTA 2','PREGUNTA 3','PREGUNTA 4','PREGUNTA 5','PREGUNTA 6','PREGUNTA 7','PREGUNTA 8'])

df2M['DNI']=df_anon_unicos['DNI'].astype(int)
df2M['COMISION']=df_anon_unicos['COMISION'].astype(str)
df2M['CARRERA']=df_anon_unicos['Carrera-Nom'].astype(str)

df2B['DNI']=df_anon_unicos['DNI'].astype(int)
df2B['COMISION']=df_anon_unicos['COMISION'].astype(str)
df2B['CARRERA']=df_anon_unicos['Carrera-Nom'].astype(str)


archivos_en_carpeta=os.listdir(f"Actividades de Seguimiento/Actividad de Seguimiento 2")
for dni in df2M.DNI:
#for dni in [41449081]:
    com=df2M['COMISION'][df2M.DNI==dni].values[0]
    letra_comision=com[0]
    for archivo in archivos_en_carpeta:
        if str(com) in archivo:
            if letra_comision=='M':
                df_a2M=pd.read_excel(f"Actividades de Seguimiento/Actividad de Seguimiento 2/{archivo}")
                mask=df_a2M['Número de ID']==dni
                
                resultM = df_a2M[mask].iloc[:,11:].values
                
                if resultM.shape[0] > 0:
                    filaM=resultM[0]
                    fila_numerosM=np.array([float(valor.replace(',', '.'))if valor.replace(',','.').replace('.','',1).isdigit() else np.nan for valor in resultM[0]])
                    df2M.iloc[df2M.DNI==dni,3:] = fila_numerosM
            

            if letra_comision=='B':
                df_a2B=pd.read_excel(f"Actividades de Seguimiento/Actividad de Seguimiento 2/{archivo}")
                mask=df_a2B['Número de ID']==dni
                
                resultB= df_a2B[mask].iloc[:,11:].values
                
                if resultB.shape[0] > 0:
                    filaB=resultB[0]
                    fila_numerosB=np.array([float(valor.replace(',', '.'))if valor.replace(',','.').replace('.','',1).isdigit() else np.nan for valor in resultB[0]])
                    df2B.iloc[df2B.DNI==dni,3:] = fila_numerosB

In [ ]:
#VISUALIZACION DE DATOS DE LA ACTIVIDAD DE SEGUIMIENTO 2 COMISION M
print('COMISIÓN M---------------------------------------------------------')
print("Puntuacion promedio de cada pregunta en la Actividad de Seguimiento 2:")
promedios=[]
topes2M=[]
for num in range(1,9 ):
    promedio = float(df2M[f'PREGUNTA {num}'].mean())
    promedios.append(round(promedio, 2))
    valor = num + 10
    # Extraer valor_max del nombre de la columna por ejemplo: 'P. 1 /0,75'
    nombre_columna = df_a2M.columns[valor]
    if '/' in nombre_columna:
        valor_max = float(nombre_columna.split('/')[1].replace(',', '.'))
        topes2M.append(valor_max)
    
    print(f"PREGUNTA {num}  :  {promedio:.2f} de {valor_max:.2f}")

#Histograma de las puntuaciones de cada pregunta en relacion al valor máximo de la pregunta
razon=[]
preguntas=[]
print(f"Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de seguimiento 2):")   
for indice in range(1, 9):
     razon.append(promedios[indice - 1] / topes2M[indice - 1])
     lugar=indice+2
     preguntas.append(f'Pregunta {indice} ')
     
     print(f"PREGUNTA {indice} : {razon[indice - 1]:.2f} ")


plt.bar(preguntas, razon, color='blue', edgecolor='black')
plt.suptitle('Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de Seguimiento 2)')
plt.title('Comisión M')

plt.ylabel('Puntuación promedio / Valor máximo')
plt.xticks(rotation=45)
plt.grid(True, axis='y', alpha=0.3)
plt.show()

In [ ]:
#ACTIVIDAD DE SEGUIMIENTO 2 POR CARRERA COMISION M---------------------------------------------------------------
print('ACTIVIDAD DE SEGUIMIENTO 2 POR CARRERA (COMISIÓN M) ------------------------------------------------------------------------------------------------')
print('')
for carrera in df2M['CARRERA'].unique():
    
    mask=df2M['CARRERA']==carrera
    maskM=mask & (df2M['COMISION'].str.startswith('M'))  # Asegurarse de que sea la comisión M  
    filas_true=df2M[maskM]

    print(f'CARRERA: {carrera}')


    if len(filas_true) <= 50:
        if len(filas_true) == 0:
            print(f'La carrera {carrera} no tiene alumnos en la comisión M.')
            
        else:
            
            print(f"No se analiza esta carrera porque tiene {len(filas_true)} alumnos (menos de 51).")
        continue


    result = filas_true.iloc[:, 3:].values #con esto obtengo todas las filas que cumplen la condicion de la carrera
   
    
    print(f'Puntuación promedio de cada pregunta:')
    
        # Verificar que result no esté vacío
    if result.size > 0:
            result = result.astype(float)
            promedios = np.nanmean(result, axis=0)
            
            # Imprimir los promedios
          
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"PREGUNTA {i}: No hay datos válidos (todos son nan)")
                else:
                    print(f"PREGUNTA {i}: {promedio:.2f}")
            
            print('')
            print('Puntuación promedio de cada pregunta en relación a su valor máximo:')
            razones=[]
            preguntas=[]
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"Razón PREGUNTA {i}: No se puede calcular (sin datos válidos)")
                else:
                    razon = promedio / topes2M[i-1]
                    print(f"Razón PREGUNTA {i}: {razon:.2f}")
                    razones.append(razon)
                    preguntas.append(f"PREGUNTA {i}")
        
        # Generar gráfico de barras para las razones
            if razones:  # Solo generar el gráfico si hay razones válidas
                plt.figure(figsize=(10, 6))
                plt.bar(preguntas, razones, color='orange',edgecolor='forestgreen',alpha=0.5)
                
                plt.ylabel('Razón (Promedio / Valor Máximo)')
                plt.title(f'Razones por Pregunta para la Carrera {carrera}')
                plt.suptitle('Actividad de seguimiento 2 ')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            else:
                print("No hay razones válidas para generar el gráfico.")
            

    else:
        print("No hay filas para esta carrera.")

    print('')


            

            



In [ ]:
#VISUALIZACION DE DATOS DE LA ACTIVIDAD DE SEGUIMIENTO 2 COMISION B
print('COMISIÓN B---------------------------------------------------------')
print("Puntuacion promedio de cada pregunta en la Actividad de Seguimiento 2:")
promedios=[]
topes2B=[]
for num in range(1,9 ):
    promedio = float(df2B[f'PREGUNTA {num}'].mean())
    promedios.append(round(promedio, 2))
    valor = num + 10
    # Extraer valor_max del nombre de la columna por ejemplo: 'P. 1 /0,75'
    nombre_columna = df_a2B.columns[valor]
    if '/' in nombre_columna:
        valor_max = float(nombre_columna.split('/')[1].replace(',', '.'))
        topes2B.append(valor_max)
    
    print(f"PREGUNTA {num}  :  {promedio:.2f} de {valor_max:.2f}")

#Histograma de las puntuaciones de cada pregunta en relación al valor máximo de la pregunta
razon=[]
preguntas=[]
print(f"Puntuacion promedio de cada pregunta en relación a su valor máximo (Actividad de seguimiento 2):")
for indice in range(1, 9):
     razon.append(promedios[indice - 1] / topes2B[indice - 1])
     lugar=indice+2
     preguntas.append(f'Pregunta {indice} ')
     
     print(f"PREGUNTA {indice} : {razon[indice - 1]:.2f} ")


plt.bar(preguntas, razon, color='orange', edgecolor='orange')
plt.suptitle('Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de Seguimiento 2)')
plt.title('Comisión B')

plt.ylabel('Puntuación promedio / Valor máximo')
plt.xticks(rotation=45)
plt.grid(True, axis='y', alpha=0.3)
plt.show()

In [ ]:
#ACTIVIDAD DE SEGUIMIENTO 2 POR CARRERA COMISION B---------------------------------------------------------------
print('ACTIVIDAD DE SEGUIMIENTO 2 POR CARRERA (COMISION B) ------------------------------------------------------------------------------------------------')
print('')
for carrera in df2B['CARRERA'].unique():
    
    mask=df2B['CARRERA']==carrera
    maskB=mask & (df2B['COMISION'].str.startswith('B'))  # Asegurarse de que sea la comisión M  
    filas_true=df2B[maskB]

    print(f'CARRERA: {carrera}')


    if len(filas_true) <= 50:
        if len(filas_true) == 0:
            print(f'La carrera {carrera} no tiene alumnos en la comisión B.')
            
        else:
            
            print(f"No se analiza esta carrera porque tiene {len(filas_true)} alumnos (menos de 51).")
        continue


    result = filas_true.iloc[:, 3:].values #con esto obtengo todas las filas que cumplen la condicion de la carrera
   
    
    print(f'Puntuación promedio de cada pregunta:')
    
        # Verificar que result no esté vacío
    if result.size > 0:
            result = result.astype(float)
            promedios = np.nanmean(result, axis=0)
            
            # Imprimir los promedios
          
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"PREGUNTA {i}: No hay datos válidos (todos son nan)")
                else:
                    print(f"PREGUNTA {i}: {promedio:.2f}")
            
            print('')
            print('Puntuación promedio de cada pregunta en relación a su valor máximo:')
            razones=[]
            preguntas=[]
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"Razón PREGUNTA {i}: No se puede calcular (sin datos válidos)")
                else:
                    razon = promedio / topes2B[i-1]
                    print(f"Razón PREGUNTA {i}: {razon:.2f}")
                    razones.append(razon)
                    preguntas.append(f"PREGUNTA {i}")
        
        # Generar gráfico de barras para las razones
            if razones:  # Solo generar el gráfico si hay razones válidas
                plt.figure(figsize=(10, 6))
                plt.bar(preguntas, razones, color='orange',edgecolor='forestgreen',alpha=0.5)
                plt.xlabel('Preguntas')
                plt.ylabel('Razón (Promedio / Valor Máximo)')
                plt.title(f'Razones por Pregunta para la Carrera {carrera}')
                plt.suptitle('Actividad 2 - Comision B')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            else:
                print("No hay razones válidas para generar el gráfico.")
            

    else:
        print("No hay filas para esta carrera.")

    print('')


            

            




In [46]:
#ANALISIS DE LA ACTIVIDAD DE SEGUIMIENTO 3 COMISIONES M Y B SEPARADAS---------------------------------------------------------


df3M=pd.DataFrame(columns=['DNI','COMISION','CARRERA','PREGUNTA 1','PREGUNTA 2','PREGUNTA 3','PREGUNTA 4','PREGUNTA 5','PREGUNTA 6'])
df3B=pd.DataFrame(columns=['DNI','COMISION','CARRERA','PREGUNTA 1','PREGUNTA 2','PREGUNTA 3','PREGUNTA 4','PREGUNTA 5','PREGUNTA 6'])

df3M['DNI']=df_anon_unicos['DNI'].astype(int)
df3M['COMISION']=df_anon_unicos['COMISION'].astype(str)
df3M['CARRERA']=df_anon_unicos['Carrera-Nom'].astype(str)

df3B['DNI']=df_anon_unicos['DNI'].astype(int)
df3B['COMISION']=df_anon_unicos['COMISION'].astype(str)
df3B['CARRERA']=df_anon_unicos['Carrera-Nom'].astype(str)


archivos_en_carpeta=os.listdir(f"Actividades de Seguimiento/Actividad de Seguimiento 3")
for dni in df3M.DNI:
#for dni in [41449081]:
    com=df3M['COMISION'][df3M.DNI==dni].values[0]
    letra_comision=com[0]
    for archivo in archivos_en_carpeta:
        if str(com) in archivo:
            if letra_comision=='M':
                df_a3M=pd.read_excel(f"Actividades de Seguimiento/Actividad de Seguimiento 3/{archivo}")
                mask=df_a3M['Número de ID']==dni
                
                resultM = df_a3M[mask].iloc[:,11:].values
                
                if resultM.shape[0] > 0:
                    filaM=resultM[0]
                    fila_numerosM=np.array([float(valor.replace(',', '.'))if valor.replace(',','.').replace('.','',1).isdigit() else np.nan for valor in resultM[0]])
                    df3M.iloc[df3M.DNI==dni,3:] = fila_numerosM
            

            if letra_comision=='B':
                df_a3B=pd.read_excel(f"Actividades de Seguimiento/Actividad de Seguimiento 3/{archivo}")
                mask=df_a3B['Número de ID']==dni
                
                resultB= df_a3B[mask].iloc[:,11:].values
                
                if resultB.shape[0] > 0:
                    filaB=resultB[0]
                    fila_numerosB=np.array([float(valor.replace(',', '.'))if valor.replace(',','.').replace('.','',1).isdigit() else np.nan for valor in resultB[0]])
                    df3B.iloc[df3B.DNI==dni,3:] = fila_numerosB

In [ ]:
#VISUALIZACION DE DATOS DE LA ACTIVIDAD DE SEGUIMIENTO 3 COMISION M
print('COMISIÓN M---------------------------------------------------------')
print("Puntuación promedio de cada pregunta en la Actividad de Seguimiento 3:")
promedios=[]
topes3M=[]
for num in range(1,7 ):
    promedio = float(df3M[f'PREGUNTA {num}'].mean())
    promedios.append(round(promedio, 2))
    valor = num + 10
    # Extraer valor_max del nombre de la columna por ejemplo: 'P. 1 /0,75'
    nombre_columna = df_a3M.columns[valor]
    if '/' in nombre_columna:
        valor_max = float(nombre_columna.split('/')[1].replace(',', '.'))
        topes3M.append(valor_max)
    
    print(f"PREGUNTA {num}  :  {promedio:.2f} de {valor_max:.2f}")

#Histograma de las puntuaciones de cada pregunta en relacion al valár maximo de la pregunta
razon=[]
preguntas=[]
print(f"Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de seguimiento 3):")
for indice in range(1, 7):
     razon.append(promedios[indice - 1] / topes3M[indice - 1])
     lugar=indice+2
     preguntas.append(f'Pregunta {indice} ')
     
     print(f"PREGUNTA {indice} : {razon[indice - 1]:.2f} ")


plt.bar(preguntas, razon, color='blue', edgecolor='black')
plt.suptitle('Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de Seguimiento 3)')
plt.title('Comisión M')

plt.ylabel('Puntuación promedio / Valor máximo')
plt.xticks(rotation=45)
plt.grid(True, axis='y', alpha=0.3)
plt.show()

In [ ]:
#ACTIVIDAD DE SEGUIMIENTO 3 POR CARRERA  COMISION M----------------------------------------------------------------
print('ACTIVIDAD DE SEGUIMIENTO 3 POR CARRERA (COMISIÓN M) ------------------------------------------------------------------------------------------------')
print('')
for carrera in df3M['CARRERA'].unique():
    
    mask=df3M['CARRERA']==carrera
    maskM=mask & (df3M['COMISION'].str.startswith('M'))  # Asegurarse de que sea la comisión M  
    filas_true=df3M[maskM]

    print(f'CARRERA: {carrera}')


    if len(filas_true) <= 50:
        if len(filas_true) == 0:
            print(f'La carrera {carrera} no tiene alumnos en la comisión M.')
            print('')
            
        else:
            
            print(f"No se analiza esta carrera porque tiene {len(filas_true)} alumnos (menos de 51).")
        continue


    result = filas_true.iloc[:, 3:].values #con esto obtengo todas las filas que cumplen la condicion de la carrera
   
    
    print(f'Puntuación promedio de cada pregunta:')
    
        # Verificar que result no esté vacío
    if result.size > 0:
            result = result.astype(float)
            promedios = np.nanmean(result, axis=0)
            
            # Imprimir los promedios
          
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"PREGUNTA {i}: No hay datos válidos (todos son nan)")
                else:
                    print(f"PREGUNTA {i}: {promedio:.2f}")
            
            print('')
            print('Puntuación promedio de cada pregunta en relación a su valor máximo:')
            razones=[]
            preguntas=[]
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"Razón PREGUNTA {i}: No se puede calcular (sin datos válidos)")
                else:
                    razon = promedio / topes3M[i-1]
                    print(f"Razón PREGUNTA {i}: {razon:.2f}")
                    razones.append(razon)
                    preguntas.append(f"PREGUNTA {i}")
        
        # Generar gráfico de barras para las razones
            if razones:  # Solo generar el gráfico si hay razones válidas
                plt.figure(figsize=(10, 6))
                plt.bar(preguntas, razones, color='BROWN',edgecolor='forestgreen',alpha=0.3)
                
                plt.ylabel('Razón (Promedio / Valor Máximo)')
                plt.title(f'Razones por Pregunta para la Carrera {carrera}')
                plt.suptitle('Actividad de seguimiento 3')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            else:
                print("No hay razones válidas para generar el gráfico.")
            

    else:
        print("No hay filas para esta carrera.")

    print('')


            

            



In [ ]:
#VISUALIZACION DE DATOS DE LA ACTIVIDAD DE SEGUIMIENTO 3 COMISION B
print('COMISIÓN B---------------------------------------------------------')
print("PuntuaciÓn promedio de cada pregunta en la Actividad de Seguimiento 3:")
promedios=[]
topes3B=[]
for num in range(1,7 ):
    promedio = float(df3B[f'PREGUNTA {num}'].mean())
    promedios.append(round(promedio, 2))
    valor = num + 10
    # Extraer valor_max del nombre de la columna por ejemplo: 'P. 1 /0,75'
    nombre_columna = df_a3B.columns[valor]
    if '/' in nombre_columna:
        valor_max = float(nombre_columna.split('/')[1].replace(',', '.'))
        topes3B.append(valor_max)
    
    print(f"PREGUNTA {num}  :  {promedio:.2f} de {valor_max:.2f}")

#Histograma de las puntuaciones de cada pregunta en relación al valor maximo de la pregunta
razon=[]
preguntas=[]
print(f"PuntuaciÓn promedio de cada pregunta en relación a su valor máximo (Actividad de seguimiento 3):")
for indice in range(1, 7):
     razon.append(promedios[indice - 1] / topes3B[indice - 1])
     lugar=indice+2
     preguntas.append(f'Pregunta {indice} ')
     
     print(f"PREGUNTA {indice} : {razon[indice - 1]:.2f} ")


plt.bar(preguntas, razon, color='orange', edgecolor='orange')
plt.suptitle('Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de Seguimiento 3)')
plt.title('Comisión B')
plt.ylabel('Puntuación promedio / Valor máximo')
plt.xticks(rotation=45)
plt.grid(True, axis='y', alpha=0.3)
plt.show()

In [ ]:
#ACTIVIDAD DE SEGUIMIENTO 3 POR CARRERA COMISION B----------------------------------------------------------------
print('ACTIVIDAD DE SEGUIMIENTO 3 POR CARRERA (COMISIÓN B) ------------------------------------------------------------------------------------------------')
print('')
for carrera in df3B['CARRERA'].unique():
    
    mask=df3B['CARRERA']==carrera
    maskB=mask & (df3B['COMISION'].str.startswith('B'))  # Asegurarse de que sea la comisión B
    filas_true=df3B[maskB]

    print(f'CARRERA: {carrera}')


    if len(filas_true) <= 50:
        if len(filas_true) == 0:
            print(f'La carrera {carrera} no tiene alumnos en la comisión B.')
            print('')
            
        else:
            
            print(f"No se analiza esta carrera porque tiene {len(filas_true)} alumnos (menos de 51).")
        continue


    result = filas_true.iloc[:, 3:].values #con esto obtengo todas las filas que cumplen la condicion de la carrera
   
    
    print(f'Puntuación promedio de cada pregunta:')
    
        # Verificar que result no esté vacío
    if result.size > 0:
            result = result.astype(float)
            promedios = np.nanmean(result, axis=0)
            
            # Imprimir los promedios
          
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"PREGUNTA {i}: No hay datos válidos (todos son nan)")
                else:
                    print(f"PREGUNTA {i}: {promedio:.2f}")
            
            print('')
            print('Puntuación promedio de cada pregunta en relación a su valor máximo:')
            razones=[]
            preguntas=[]
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"Razón PREGUNTA {i}: No se puede calcular (sin datos válidos)")
                else:
                    razon = promedio / topes3B[i-1]
                    print(f"Razón PREGUNTA {i}: {razon:.2f}")
                    razones.append(razon)
                    preguntas.append(f"PREGUNTA {i}")
        
        # Generar gráfico de barras para las razones
            if razones:  # Solo generar el gráfico si hay razones válidas
                plt.figure(figsize=(10, 6))
                plt.bar(preguntas, razones, color='BROWN',edgecolor='forestgreen',alpha=0.3)
                plt.xlabel('Preguntas')
                plt.ylabel('Razón (Promedio / Valor Máximo)')
                plt.title(f'Razones por Pregunta para la Carrera {carrera}')
                plt.suptitle('Actividad 3 ')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            else:
                print("No hay razones válidas para generar el gráfico.")
            

    else:
        print("No hay filas para esta carrera.")

    print('')


            

            



In [41]:
#ANALISIS DE LA ACTIVIDAD DE SEGUIMIENTO 4 COMISION M---------------------------------------------------------


df4M=pd.DataFrame(columns=['DNI','COMISION','CARRERA','PREGUNTA 1','PREGUNTA 2','PREGUNTA 3','PREGUNTA 4','PREGUNTA 5','PREGUNTA 6','PREGUNTA 7'])


df4M['DNI']=df_anon_unicos['DNI'].astype(int)
df4M['COMISION']=df_anon_unicos['COMISION'].astype(str)
df4M['CARRERA']=df_anon_unicos['Carrera-Nom'].astype(str)


archivos_en_carpeta=os.listdir(f"Actividades de Seguimiento/Actividad de Seguimiento 4")
for dni in df4M.DNI:
#for dni in [41449081]:
    com=df4M['COMISION'][df4M.DNI==dni].values[0]
    letra_comision=com[0]
    for archivo in archivos_en_carpeta:
        if str(com) in archivo:
            if letra_comision=='M':
                df_a4M=pd.read_excel(f"Actividades de Seguimiento/Actividad de Seguimiento 4/{archivo}")
                mask=df_a4M['Número de ID']==dni
                
                resultM = df_a4M[mask].iloc[:,11:].values
                
                if resultM.shape[0] > 0:
                    filaM=resultM[0]
                    fila_numerosM=np.array([float(valor.replace(',', '.'))if valor.replace(',','.').replace('.','',1).isdigit() else np.nan for valor in resultM[0]])
                    df4M.iloc[df4M.DNI==dni,3:] = fila_numerosM
            

            

In [ ]:
#VISUALIZACION DE DATOS DE LA ACTIVIDAD DE SEGUIMIENTO 4 COMISION M
print('COMISION M---------------------------------------------------------')
print("Puntuacion promedio de cada pregunta en la Actividad de Seguimiento 4:")
promedios=[]
topes4M=[]
for num in range(1,8 ):
    promedio = float(df4M[f'PREGUNTA {num}'].mean())
    promedios.append(round(promedio, 2))
    valor = num + 10
    # Extraer valor_max del nombre de la columna por ejemplo: 'P. 1 /0,75'
    nombre_columna = df_a4M.columns[valor]
    if '/' in nombre_columna:
        valor_max = float(nombre_columna.split('/')[1].replace(',', '.'))
        topes4M.append(valor_max)
    
    print(f"PREGUNTA {num}  :  {promedio:.2f} de {valor_max:.2f}")

#Histograma de las puntuaciones de cada pregunta en relacion al valor maximo de la pregunta
razon=[]
preguntas=[]
print(f"Puntuacion promedio de cada pregunta en relacion a su valor maximo (Actividad de seguimiento 4):")
for indice in range(1, 8):
     razon.append(promedios[indice - 1] / topes4M[indice - 1])
     lugar=indice+2
     preguntas.append(f'Pregunta {indice} ')
     
     print(f"PREGUNTA {indice} : {razon[indice - 1]:.2f} ")


plt.bar(preguntas, razon, color='blue', edgecolor='black')
plt.suptitle('Puntuación promedio de cada pregunta en relación a su valor máximo (Actividad de Seguimiento 4)')
plt.title('Comisión M')

plt.ylabel('Puntuación promedio / Valor máximo')
plt.xticks(rotation=45)
plt.grid(True, axis='y', alpha=0.3)
plt.show()

In [ ]:
#ACTIVIDAD DE SEGUIMIENTO 4 POR CARRERA  COMISION M----------------------------------------------------------------
print('ACTIVIDAD DE SEGUIMIENTO 4 POR CARRERA (COMISION M) ------------------------------------------------------------------------------------------------')
print('')
for carrera in df4M['CARRERA'].unique():
    
    mask=df4M['CARRERA']==carrera
    maskM=mask & (df4M['COMISION'].str.startswith('M'))  # Asegurarse de que sea la comisión M  
    filas_true=df4M[maskM]

    print(f'CARRERA: {carrera}')


    if len(filas_true) <= 50:
        if len(filas_true) == 0:
            print(f'La carrera {carrera} no tiene alumnos en la comisión M.')
            print('')
            
        else:
            
            print(f"No se analiza esta carrera porque tiene {len(filas_true)} alumnos (menos de 51).")
        continue


    result = filas_true.iloc[:, 3:].values #con esto obtengo todas las filas que cumplen la condicion de la carrera
   
    
    print(f'Puntuación promedio de cada pregunta:')
    
        # Verificar que result no esté vacío
    if result.size > 0:
            result = result.astype(float)
            promedios = np.nanmean(result, axis=0)
            
            # Imprimir los promedios
          
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"PREGUNTA {i}: No hay datos válidos (todos son nan)")
                else:
                    print(f"PREGUNTA {i}: {promedio:.2f}")
            
            print('')
            print('Puntuación promedio de cada pregunta en relación a su valor máximo:')
            razones=[]
            preguntas=[]
            for i, promedio in enumerate(promedios, 1):
                if np.isnan(promedio):
                    print(f"Razón PREGUNTA {i}: No se puede calcular (sin datos válidos)")
                else:
                    razon = promedio / topes4M[i-1]
                    print(f"Razón PREGUNTA {i}: {razon:.2f}")
                    razones.append(razon)
                    preguntas.append(f"PREGUNTA {i}")
        
        # Generar gráfico de barras para las razones
            if razones:  # Solo generar el gráfico si hay razones válidas
                plt.figure(figsize=(10, 6))
                plt.bar(preguntas, razones, color='purple',edgecolor='forestgreen',alpha=0.5)
                
                plt.ylabel('Razón (Promedio / Valor Máximo)')
                plt.title(f'Razones por Pregunta para la Carrera {carrera}')
                plt.suptitle('Actividad de seguimiento 4')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
            else:
                print("No hay razones válidas para generar el gráfico.")
            

    else:
        print("No hay filas para esta carrera.")

    print('')


            

            

